In [ ]:
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint


from keys import Trip_advisor_key
from keys import geoapify_key

In [ ]:
# Defining the District dataframe

Districts_df = pd.DataFrame({
    'Location': ['Toronto Distillery District', 'Toronto GTA Area', 'Toronto South West', 'Toronto North West', 'Toronto Yorkville', 'Ottawa City Centre' , 'Niagara Falls'],
    'Lat': [43.6503, 43.6426, 43.642567, 43.6552, 43.6707 , 45.4236, 43.087586], 
    'Lng': [-79.3596, 79.3871, -79.387054, -79.4139,  -79.3930 , -75.7009, -79.081206]  
})


# Defining the Restaurant DataFrame

Restaurants_df = pd.DataFrame(columns=['Location', 'Restaurant Name', 'Postal Code', 'Restaurant Address', 'Lat', 'Lng'])

# Loop for districts
radius = 1000

params = {
    'categories': 'catering.restaurant',
    'apiKey': geoapify_key,
}

base_url = 'https://api.geoapify.com/v2/places'

for index, row in Districts_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']

    params['filter'] = f'circle:{longitude},{latitude},{radius}'
    params['bias'] = f'proximity:{longitude},{latitude}'

    response = requests.get(base_url, params=params).json()
    for restaurant in response.get('features', []):
        try:
            
            new_row = {
                            'Location': row['Location'],
                            'Restaurant Name': restaurant["properties"]["address_line1"],
                            'Postal Code' : restaurant["properties"]["postcode"],
                            'Restaurant Address': restaurant["properties"]["address_line2"],
                            'Lat': restaurant["geometry"]["coordinates"][1],
                            'Lng': restaurant["geometry"]["coordinates"][0]
                        }
            
            Restaurants_df.loc[len(Restaurants_df)] = new_row
        except(KeyError, IndexError):
            print('Errore nella gestione dei dati del ristorante.')


In [ ]:
Restaurants_df.head()

In [ ]:
Restaurants_df['Location ID'] = ''

err_count = 0

for index, restaurant in Restaurants_df.iterrows():
    
    url = f"https://api.content.tripadvisor.com/api/v1/location/search?key={Trip_advisor_key}&searchQuery={restaurant['Restaurant Name']}&language=en"

    headers = {"accept": "application/json"}

    response = requests.get(url, headers=headers).json()
    
    try:
        
        for res in response['data']:
            
            #pprint(res['address_obj']['postalcode'])
    
            if res['address_obj']['postalcode'] == restaurant['Postal Code']:
            
                Restaurants_df.at[index, 'Location ID'] = res['location_id']
                
                break
    
    except Exception as e:
        
        err_count = err_count +1
        
        
            
    
        

#print(response.text)
print(err_count)
Restaurants_df.head()

In [ ]:
Restaurants_df = Restaurants_df[Restaurants_df['Location ID'] != '']

In [ ]:
# Export the City_Data into a csv
Restaurants_df.to_csv("output_data/restaurants.csv", index_label="Restaurant ID")

In [ ]:
Restaurants_df